In [1]:
import pandas as pd
import numpy as np
import re
from homologaciones.exportadores import exportadores_dict, importadores_dict

In [2]:
imports = pd.read_csv('matched_imports.csv')

In [3]:
ruta_medicamentos = 'C:/Users/juan-/Downloads/pharma/Bases/20220516_MEDICAMENTOS ONCOLOGICOS BIOLOGICOS_PW_AP_PW.xlsx'
MedicamentosDB = pd.read_excel(ruta_medicamentos, sheet_name='BIOLOGICOS')

## Importadores y Exportadores Homologación.

In [24]:
importadores = imports['Importador'].value_counts()
importadores = sorted(list(dict(importadores).keys()))
len(importadores)

200

In [ ]:
exportadores = imports['Exportador'].value_counts()
exportadores = sorted(list(dict(exportadores).keys()))
exportadores

## funciones

### funciones de normalizacion y homologacion

In [9]:
def homologar_importador(importador: str) -> str:
    """"""
    try:
        importador_h = importadores_dict[importador] if importador != 'sin informacion' else 'sin informacion'
    except Exception as e:
        print("error en: ",importador," con el error", e)
        importador_h = 'Error'
    
    return importador_h

In [4]:
def homologar_exportador(exportador: str) -> str:
    """"""
    try:
        exportador_h = exportadores_dict[exportador] if exportador != 'sin informacion' else 'sin informacion'
    except Exception as e:
        print("error en: ",exportador," con el error", e)
        exportador_h = 'Error'
    
    return exportador_h

In [5]:
def normalizar(text: str) -> str:
    """retorna la cadena del parametro sin acentos, en minuscula y quita espacios al principio o fin de la cadena"""
    a,b = 'áéíóúüÁÉÍÓÚÜ','aeiouuAEIOUU'
    trans = str.maketrans(a,b)
    try:
        text_normalized = text.translate(trans).lower().strip()
    except:
        print(f"error normalizando: {text}")
        text_normalized = "Error"
    return text_normalized

### funciones de clasificacion

In [7]:
def calc_valor_unitario(valor_total, unidades):
  """calcular valor_unitario dividiendo valor_total entre el número de unidades(previamente encontrado en descripcion comercial)
     return valor_unitario"""
  pass

def comprobar():
  """
    calc_valor_unitario(valor_total, unidades)
    este nuevo valor_unitario se debe contrastar contra el valor_unitario de medicamentos de la misma marca para
    verificar que no sea un outlier:
      TipoI = producto_terminado
      si es un outlier se clasificará como:
        TipoII = EstandarReferencia
      si no es outlier
        TipoII = Valor comercial
  """
  pass

def buscar_pres_medi_en_desc_comer():
  """buscar presentacion de medicamento en descripcion comercial:
      buscar unidades, tipo_unidad, concentracion, unidad_de_concentracion 
      en la descripcion_comercial.
      
      return unidad, tipo_unidad, concentracion, unidad_de_concentracion(25,pzas,50,mg) """
  pass 

def comprobar_principio_activo():
  """
  buscar_pres_medi_en_desc_comer()
  si estas están en su descripcion_comercial entonces no son PrincipioActivo AÚN
    return False
  si estas NO están en su descripcion_comercial:
    entonces podemos clasificarlas como PrincipioActivo
    return True
  """
  pass

def clasificar(row):
    """
    recibe cada fila del dataframe en su columna "descripcion_comercial",
    tratando de tipificarla:
    1)
    si estandar de referencia en descripcion_comercial
      tipo II = estander de referencia
    si unidad es igual a KG,G,L,MLL
        sí comprobar_principio_activo():
          tipo I = principio activo
        sí no:
          2) comrpobar_si_ER_o_VC()
    2)
         
    """
    pass
    

In [6]:
def clasify_er_t2_by_keywords(desc_comer : str) -> str:
    """looks for keywords in descripcion_comercial column, if the keywords are
    contained is clasify as estandar de referencia(er) on its tipoII"""
    if ("estandar de referencia" in desc_comer) or ("sustancia de referencia" in desc_comer) or ("reference standard" in desc_comer) or \
       ("estandares de refrencia" in desc_comer)  or ("muestra" in desc_comer)  or ("placebo" in desc_comer): 
        return "ER"
    else:
        return None

In [7]:
def clasify_pa_t1_by_tipounidad(t_u : str) -> str:
    """looks in its tipo_unidad columns if g,kg,l,ml are contained
    it will be clasified as principio_activo(PA) on its tipoI"""
    if ('G'==t_u) or ('KG'==t_u) or ('L'==t_u) or ('ML'==t_u):
        return 'PA'
    else:
        return None

### funcion de llenado de datos

In [161]:
#MedicamentosDB['FORMA FARMA'].value_counts() #UNIDAD CONCENTRACION(mg,µG)	FORMA FARMA(Solución Inyectable,Comprimidos, Tabletas, Capsulas, Solución oral ) 
#caja, frasco, ampula, frasco ampula
UNIDAD_CONCENTRACION = ['mg','µG']
FORMA_FARMA = ['tabletas', 'capsulas','comprimidos','solucion inyectable', 'solucion oral']

In [13]:
def get_concentracion_from_desc_comer(row):
    desc_comer = row['descripcion_comercial']
    #------------------------------4------------------------------#
    #ninlaro (ixazomib) caja colectiva con 3 cajas con una capsula de 3.5 mg (medicamento de uso humano) 
    expresion_0 = '\d\s*cajas con una capsula de \d+\.?\d*\s*mg'
    #afinitor 10 mg  30 comprimidos (everolimus)
    expresion_1 = '\d+\.?\d*\s*mg\s*\d{1,3}\s*(tabletas|tab|capsulas|comprimidos|solucion inyectable|solucion oral|ge)'
    #giotrif 30mg (afatinib) 30 tabletas (medicamento de uso humano)
    expresion_1_1 = '\d+\.?\d*\s*mg.{0,40}\d{1,4}\s*(tabletas|capsulas|comprimido|solucion inyectable|solucion oral)' #menos exacta que la expresion_1
    #prycel (dasatinib) frasco con  60 tabletas de 70 mg (medicamento de uso humano)
    expresion_2 = '\d{1,3}\s*(tablet|capsulas|comprimidos|solucion inyectable|viales|solucion oral).{0,35}\d+\.?\d*\s*(mg|gramos)'#se modificó para que detecte gramos y viales
    #tabletas.{0,5}\d+\.?\d*\smg.{0,5}\d+
    expresion_2_2 = 'tabletas.{0,5}\d+\.?\d*\smg.{0,5}\d+'#Nueva
    #------------------------------3------------------------------#
    #avastin  100 mg/4 ml solucion inyectable (bevacizumab)
    expresion_soluciones = '\d+\.?\d*\s*mg.{0,30}\s*(solucion inyectable|solucion oral)'
    #mvasi (bevacizumab) ampula con 100 mg/ 4.0 ml (medicamento de uso humano)
    expresion_4 = '(frasco ampula|vial|ampula|jeringa)\s*.{0,15}\d+\.?\d*\s*mg'
    #ninlaro (ixazomib) caja colectiva con 3 cajas con una capsula de 2.3 mg (medicamento de uso humano)
    expresion_4_2 = '(1|un|una)*\s*(capsu|capsula|tabletas).{0,5}\d+\.?\d*\s*mg'
    #kadcyla 100mg frasco ampula (trastuzumab emtansina)
    expresion_6 = '\d+\.?\d*\s*mg.{0,18}(frasco ampula|vial|ampula|jeringa)'
    #medicamento de uso humano vitrakvi (larotrectinib 100mg)capsulas
    expresion_7 = '\d+\.?\d*\s*mg\)*capsulas'
    #tarceva 100 mg comprimidos  (erlotinib)
    expresion_3 = '\d+\.?\d*\s*mg\s*(tabletas|tablet|capsul|comprimidos|solucion inyectable|solucion oral)'
    #------------------------------2------------------------------#
    #kyprolis 60mg (carfilzomib) (medicamento de uso humano)
    expresion_5 = '\d+\.?\d*\s*mg'
    #vial solucion estabilizadora iv para blinatumomab caja con 12 viales
    expresion_5_2 = '\d+\s*(tabletas|tab|capsulas|capsula|comprimidos|solucion inyectable|solucion oral|vial)'
    #brigatinib (alunbring)(taqbletas)
    expresion_s_ff = '(tabletas|tab|capsulas|capsula|comprimidos|solucion inyectable|solucion oral|vial)'

    if not isinstance(desc_comer, str):
        return [np.nan, np.nan,np.nan, np.nan]
     #----------------------------------------4--------------------------------------#
    match = re.search(expresion_0, desc_comer)#'\d+\.?\d*\s*mg\s*\d{1,3}\s*(tabletas|tab|capsulas|comprimidos|solucion inyectable|solucion oral)'
    if match:#10 mg  30 comprimidos
        info = match.group()
        c_uc =  re.search("\d+\.?\d*\s*mg", info).group()
        concentracion = re.search("\d+\.?\d*", c_uc).group()
        unidad_concentracion = "mg"
        p_ff = re.search("\d\s*cajas", info).group()
        presentacion =  re.search("\d{1,3}", p_ff).group()
        forma_farma = 'capsulas'
        return [concentracion, unidad_concentracion, presentacion, forma_farma]

    match = re.search(expresion_1, desc_comer)#'\d+\.?\d*\s*mg\s*\d{1,3}\s*(tabletas|tab|capsulas|comprimidos|solucion inyectable|solucion oral)'
    if match:#10 mg  30 comprimidos
        info = match.group()
        c_uc =  re.search("\d+\.?\d*\s*mg", info).group()
        concentracion = re.search("\d+\.?\d*", c_uc).group()
        unidad_concentracion = "mg"
        p_ff = re.search("\d{1,3}\s*(tabletas|tab|capsulas|comprimidos|solucion inyectable|solucion oral|ge)", info).group()
        presentacion =  re.search("\d{1,3}", p_ff).group()
        forma_farma = re.search("(tabletas|tab|capsulas|comprimidos|solucion inyectable|solucion oral|ge)", p_ff).group()
        forma_farma = 'tabletas' if forma_farma == 'tab' else forma_farma
        forma_farma = 'sff' if forma_farma == 'ge' else forma_farma
        return [concentracion, unidad_concentracion, presentacion, forma_farma]

    match = re.search(expresion_1_1, desc_comer)#'\d+\.?\d*\s*mg.{0,40}\d{1,4}\s*(tabletas|capsulas|comprimidos|solucion inyectable|solucion oral)'
    if match:#100 mg/4 ml solucion inyectable
        info = match.group()
        c_uc =  re.search("\d+\.?\d*\s*mg", info).group()
        concentracion = re.search("\d+\.?\d*", c_uc).group()
        unidad_concentracion = "mg"
        p_ff = re.search("\d{1,4}\s*(tabletas|capsulas|comprimido|solucion inyectable|solucion oral)", info).group()
        presentacion =  re.search("\d{1,4}", p_ff).group()
        forma_farma = re.search("(tabletas|capsulas|comprimido|solucion inyectable|solucion oral)", p_ff).group()
        forma_farma = 'comprimidos' if  'comrpimido' in forma_farma else forma_farma
        return [concentracion, unidad_concentracion, presentacion, forma_farma]

    match = re.search(expresion_2, desc_comer)#'\d{1,3}\s*(tablet|capsulas|comprimidos|solucion inyectable|viales|solucion oral).{0,35}\d+\.?\d*\s*(mg|gramos)'
    if match:#60 capsulas de 250mg
        info = match.group()
        c_uc =  re.search("\d+\.?\d*\s*(mg|gramos)", info).group()
        concentracion = re.search("\d+\.?\d*", c_uc).group()
        unidad_concentracion = re.search("(mg|gramos)", c_uc).group()
        p_ff = re.search("\d{1,3}\s*(tablet|capsulas|comprimidos|solucion inyectable|viales|solucion oral)", info).group()
        presentacion = re.search("\d{1,3}", p_ff).group()
        forma_farma = re.search("(tablet|capsulas|comprimidos|solucion inyectable|viales|solucion oral)", p_ff).group()
        forma_farma = 'tabletas' if 'tablet' in forma_farma else forma_farma
        return [concentracion, unidad_concentracion, presentacion, forma_farma]

    match = re.search(expresion_2_2, desc_comer) #'tabletas.{0,5}\d+\.?\d*\smg.{0,5}\d+'
    if match:#tabletas de 250 mg con 30 
        info = match.group()
        c_uc =  re.search("\d+\.?\d*\s*mg", info).group()
        concentracion = re.search("\d+\.?\d*", c_uc).group()
        unidad_concentracion = 'mg'
        p_ff = re.search("con\s*\d{1,3}", info).group()
        presentacion = re.search("\d{1,3}", p_ff).group()
        forma_farma = 'tabletas'
        return [concentracion, unidad_concentracion, presentacion, forma_farma]
    
    #----------------------------------------3--------------------------------------#
    match = re.search(expresion_3, desc_comer)#'\d+\.?\d*\s*mg\s*(tabletas|tablet|capsul|comprimidos|solucion inyectable|solucion oral)'
    if match:#250 mg tabletas
        info = match.group()
        numeros =  re.findall("\d+\.?\d*", info)
        concentracion = numeros[0]
        unidad_concentracion = 'mg'
        presentacion = 'sp'
        forma_farma = re.search("(tabletas|tablet|capsul|comprimidos|solucion inyectable|solucion oral)", info).group()
        forma_farma = 'tabletas' if forma_farma == 'tablet' else forma_farma
        forma_farma = 'capsulas' if 'capsul' in forma_farma else forma_farma
        return [concentracion, unidad_concentracion, presentacion, forma_farma]

    match = re.search(expresion_soluciones, desc_comer)#'\d+\.?\d*\s*mg.{0,30}\s*(solucion inyectable|solucion oral)'
    if match:#100 mg/4 ml solucion inyectable
        info = match.group()
        concentracion = re.search("\d+\.?\d*", info).group()
        unidad_concentracion = 'mg'
        presentacion = '1'
        forma_farma = re.search("(tabletas|tab|capsulas|comprimidos|solucion inyectable|solucion oral)", info).group()
        return [concentracion, unidad_concentracion, presentacion, forma_farma]
    
    match = re.search(expresion_4, desc_comer)#'(frasco ampula|vial|ampula|jeringa)\s*.{0,15}\d+\.?\d*\s*mg'
    if match:#ampula con 100 mg  
        info = match.group()
        c_uc = re.search("\d+\.?\d*\s*mg", info).group()
        concentracion = re.search("\d+\.?\d*", c_uc).group()
        unidad_concentracion = 'mg'
        presentacion = '1'
        forma_farma = re.search("(frasco ampula|vial|ampula|jeringa)", info).group()
        return [concentracion, unidad_concentracion, presentacion, forma_farma]

    match = re.search(expresion_4_2, desc_comer)#'(1|un|una)*\s*(capsu|capsula|tabletas).{0,5}\d+\.?\d*\s*mg'
    if match:#una capsula de 2.3 mg
        info = match.group()
        c_uc =  re.search("\d+\.?\d*\s*mg", info).group()
        concentracion = re.search("\d+\.?\d*", c_uc).group()
        unidad_concentracion = "mg"
        p_ff = re.search("(1|un|una)*\s*(capsu|capsula|tabletas)", info).group()
        presentacion =  re.search("(1|un|una)", p_ff)
        forma_farma = re.search("(capsu|capsula|tabletas)", p_ff).group()
        forma_farma = 'capsulas' if (forma_farma == 'capsu' or forma_farma == 'capsula') else forma_farma
        presentacion = '1' if presentacion else 'sp'
        caracteristicas = 4 if presentacion == '1' else 3
        return [concentracion, unidad_concentracion, presentacion, forma_farma]

    match = re.search(expresion_6, desc_comer)#\d+\.?\d*\s*mg.{0,10}(frasco ampula|vial|ampula|jeringa)
    if match:#100mg frasco ampula #120 mg/ 2.4 ml frasco ampula
        info = match.group()
        c_uc =  re.search("\d+\.?\d*\s*mg", info).group()
        concentracion = re.search("\d+\.?\d*", c_uc).group()
        unidad_concentracion = "mg"
        presentacion = '1'
        forma_farma = re.search("(frasco ampula|vial|ampula|jeringa)", info).group()
        return [concentracion, unidad_concentracion, presentacion, forma_farma]
    
    match = re.search(expresion_7, desc_comer)#\d+\.?\d*\s*mg\)*capsulas
    if match: #100mg)capsulas
        info = match.group()
        concentracion = re.search("\d+\.?\d*", info).group()
        unidad_concentracion = 'mg'
        presentacion = 'sp'
        forma_farma = 'capsulas'
        return [concentracion, unidad_concentracion, presentacion, forma_farma]
   
    #----------------------------------------2--------------------------------------#
    match = re.search(expresion_5, desc_comer)#\d+\.?\d*\s*mg
    if match:#250 mg
        info = match.group()
        concentracion =  re.search("\d+\.?\d*", info).group()
        unidad_concentracion = 'mg'
        presentacion = 'sp'
        forma_farma = 'sff'
        return [concentracion, unidad_concentracion, presentacion, forma_farma]
    
    match = re.search(expresion_5_2, desc_comer)#\d+\s*(tabletas|tab|capsulas|capsula|comprimidos|solucion inyectable|solucion oral|vial)
    if match:#12 viales
        info = match.group()
        concentracion = 'sc'
        unidad_concentracion = 'suc'
        presentacion = re.search("\d+", info).group()
        forma_farma = re.search("(tabletas|tab|capsulas|capsula|comprimidos|solucion inyectable|solucion oral|vial)", info).group()
        forma_farma = 'tabletas' if forma_farma == 'tab' else forma_farma
        forma_farma = 'capsulas' if 'capsul' in forma_farma else forma_farma
        return [concentracion, unidad_concentracion, presentacion, forma_farma]
    
    #----------------------------------------1--------------------------------------#
    match = re.search(expresion_s_ff, desc_comer)#(tabletas|taqbletas|tab|capsulas|capsula|comprimidos|solucion inyectable|solucion oral|vial)
    if match:#taqbletas
        info = match.group()
        concentracion = 'sc'
        unidad_concentracion = 'suc'
        presentacion = 'sp'
        forma_farma = match.group()
        forma_farma = 'tabletas' if ('tab' in forma_farma or forma_farma == 'taqbletas') else forma_farma
        forma_farma = 'capsulas' if 'capsul' in forma_farma else forma_farma
        return [concentracion, unidad_concentracion, presentacion, forma_farma]

    return [np.nan, np.nan, np.nan, np.nan]

## Normalizando Importadores y Exportadores

In [10]:
imports['Importador'].fillna('sin informacion', inplace=True)
imports['Exportador'].fillna('sin informacion', inplace=True)
imports['importador_h'] = imports['Importador'].apply(homologar_importador)
imports['exportador_h'] = imports['Exportador'].apply(homologar_exportador)
#se homologa porque se me olvidó homologar antes los exportadores e importadores y al concatenarlos con los paises que vienenen normalizados se veian tal que asi: AZTRAZENECA mexico
imports['importador_h'] = imports['importador_h'].apply(normalizar)
imports['exportador_h'] = imports['exportador_h'].apply(normalizar)
#se concatena el exportador mas el pais de origen y compra respectivamente
imports['exportador_h_po'] = imports.apply(lambda row: row['exportador_h'] + ' ' + row['pais_origen_h'] if row['exportador_h'] != 'sin informacion' else 'sin informacion',axis=1)
imports['exportador_h_pc'] = imports.apply(lambda row: row['exportador_h'] + ' ' + row['pais_compra_h'] if row['exportador_h'] != 'sin informacion' else 'sin informacion',axis=1)

## Normalizando Medicamentos

In [11]:
MedicamentosDB['MEDICAMENTO'] = MedicamentosDB['MEDICAMENTO'].apply(normalizar)
MedicamentosDB['NOMBRE COMERCIAL'] = MedicamentosDB['NOMBRE COMERCIAL'].apply(normalizar)
MedicamentosDB['FORMA FARMA'] = MedicamentosDB['FORMA FARMA'].apply(normalizar)

## llenado de datos con NM

In [11]:
#creacion de columnas para ir detectando cambios y más
imports['concentracion'] = ''
imports['unidad_concentracion'] = ''
imports['cnis'] = ''
imports['status_medicamento'] = ''
imports['presentacion'] = ''
imports['forma_farma'] = ''
imports['nombre_medicamento'] = ''
imports['empresa'] = ''
imports['unidosis'] = ''
imports['tipoI'] = ''
imports['tipoII'] = ''
imports['c_r_c_llenado_datos'] = ''

## Clasificación tipo 1

In [12]:
imports['tipoII'] = imports['descripcion_comercial'].apply(clasify_er_t2_by_keywords)
imports['tipoI'] = imports['tipo_unidad'].apply(clasify_pa_t1_by_tipounidad)

In [ ]:
unidades = {
'PZA':'pieza',
'CAJ':'caja',
'BOT':'botella',
'JGO':'juego',
'L':'litro',
'MLL':'mililitro',
'G':'gramos',      
'KG':'kilogramos'
}
imports['tipo_unidad'].value_counts(dropna=False)

## LLenado de presentación farmacologica

In [14]:
imports[['concentracion','unidad_concentracion','presentacion','forma_farma']] = imports.apply(get_concentracion_from_desc_comer, axis=1, result_type='expand')

In [18]:
#full imports selected columns
imports[['tipo_unidad','unidades','descripcion_comercial','nombre','marca','concentracion','unidad_concentracion','presentacion','forma_farma','tipoI','tipoII']].to_csv('coercion.csv')

In [15]:
#imports COMPLETE only with concentracion
imports[~imports['concentracion'].isna()].to_csv('cp_complete.csv')

In [20]:
#imports only with concentracion
imports[['unidades','tipo_unidad','nombre_y_marca','nombre','marca','descripcion_comercial','presentacion','forma_farma','concentracion','unidad_concentracion','tipoI','tipoII']][~imports['concentracion'].isna()].to_csv('cp.csv')

In [21]:
#imports only without concentracion
imports[['unidades','tipo_unidad','nombre_y_marca','nombre','marca','descripcion_comercial','presentacion','forma_farma','concentracion','unidad_concentracion','tipoI','tipoII']][imports['concentracion'].isna()].to_csv('sp.csv')

In [ ]:
#sin sp sin sff y sin ningun dato vacio
imports[['forma_farma','concentracion','unidad_concentracion','presentacion']][(imports['presentacion']!='sp') & (imports['forma_farma']!='sff')&(imports['nombre_y_marca']=='NM')].dropna(how='any')

In [22]:
imports[imports['concentracion'].isna()].shape  #98

(3187, 32)